In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import re
import matplotlib as mpl

plt.rcParams["font.family"] = "Malgun Gothic"  
mpl.rcParams["axes.unicode_minus"] = False

df = pd.read_csv("2025_Airbnb_NYC_listings.csv")
df

In [ ]:
# 전처리 1 - id int로 변경
df['id'] = df['id'].astype(int)

# booking_info
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
df['instant_bookable'] =  df['instant_bookable'].map({'f':0, 't':1})
df['is_long_term'] = (df['minimum_nights'] >= 28).astype(int)

# amenities_info
def parse_amenities(x):
    try:
        return [a.strip().strip('"').strip("'") for a in ast.literal_eval(x)]
    except:
        return []
    
df['amenities'] = df['amenities'].apply(parse_amenities)
df['amenities_cnt'] = df['amenities'].apply(len)

# host_info
# neighborhood_overview 결측치 많아서 유무대체 
df['neighborhood_overview_exists'] = df['neighborhood_overview'].notnull().astype(int)

# name 글자수기준 중앙값으로 그룹
df['name_length'] = df['name'].fillna('').astype(str).apply(len)
med_length = df['name_length'].median()

def name_length_group(length, med):
    if length == 0:
        return 'empty'
    elif length > med:
        return 'long'
    else:
        return 'short_or_med'
    
df['name_length_group'] = df['name_length'].apply(lambda x: name_length_group(x, med_length))

# description 글자수기준(결측치 405) 평균으로 그룹
df['description_length'] = df['description'].fillna('').astype(str).apply(len)
avg_length = df['description_length'].mean()

def name_length_group(length, avg):
    if length == 0:
        return 'empty'
    elif length > avg:
        return 'long'
    else:
        return 'short_or_avg'
    
df['description_length_group'] = df['description_length'].apply(lambda x: name_length_group(x, avg_length))

# host_about (결측치8917) 평균(243) 중앙값(81) 중앙값기준으로 그룹
df['host_about_length'] = df['host_about'].fillna('').astype(str).apply(len)
med_length = df['host_about_length'].median()

def name_length_group(length, med):
    if length == 0:
        return 'empty'
    elif length > med:
        return 'long'
    else:
        return 'short_or_med'
df['host_about_length_group'] = df['host_about_length'].apply(lambda x: name_length_group(x, med_length))


#host_identity_verified/host_has_profile_pic /host_is_superhost  
# True / Flase 1과 0으로 대체 (결측치 20/20/350 0으로 대체함)
df['host_identity_verified']=df['host_identity_verified'].fillna('f').map({'t': 1, 'f': 0}).astype(int)

df['host_has_profile_pic']=df['host_has_profile_pic'].fillna('f').map({'t': 1, 'f': 0}).astype(int)

df['host_is_superhost']=df['host_is_superhost'].fillna('f').map({'t': 1, 'f': 0}).astype(int)

# host_response_time 결측치는 중앙값으로 치환후 점수
response_time_score_map = { 
    'within an hour': 4,
    'within a few hours': 3,
    'within a day': 2,
    'a few days or more': 1
}
df['host_response_time_score'] = df['host_response_time'].map(response_time_score_map)

# 2. response_time_score 컬럼의 중앙값 계산
med_score_for_fillna = df['host_response_time_score'].median()

# 3. response_time_score 컬럼의 NaN을 계산된 중앙값으로 대체 
df['host_response_time_score'] = df['host_response_time_score'].fillna(med_score_for_fillna)

# host_response_time 칼럼에는 여전히 nan값 존재함
# response_time_score 칼럼만 중앙값대체 


# host_response_rate 컬럼 %제외하고 중앙값으로 대체
df['host_response_rate'] = df['host_response_rate'].astype(str).str.replace('%', '').astype(float)/100
med_rate2 = df['host_response_rate'].median()
df['host_response_rate']= df['host_response_rate'].fillna(med_rate2)

# 4그룹으로 나눠 점수
conditions = [
    (df['host_response_rate'] <= 0.25),
    (df['host_response_rate'] > 0.25) & (df['host_response_rate'] <= 0.5),
    (df['host_response_rate'] > 0.5) & (df['host_response_rate'] <= 0.75),
    (df['host_response_rate'] > 0.75)
]

choices = [1, 2, 3, 4]

df['host_response_rate_score'] = np.select(conditions, choices)


# host_acceptance_rate 칼럼도 %제외하고 중앙값으로 대체 
df['host_acceptance_rate'] = df['host_acceptance_rate'].astype(str).str.replace('%', '').astype(float)/100
med_rate = df['host_acceptance_rate'].median()
df['host_acceptance_rate']= df['host_acceptance_rate'].fillna(med_rate)

conditions = [
    (df['host_acceptance_rate'] <= 0.25),
    (df['host_acceptance_rate'] > 0.25) & (df['host_acceptance_rate'] <= 0.5),
    (df['host_acceptance_rate'] > 0.5) & (df['host_acceptance_rate'] <= 0.75),
    (df['host_acceptance_rate'] > 0.75)
]

choices = [1, 2, 3, 4]

df['host_acceptance_rate_score'] = np.select(conditions, choices)

# host_location 칼럼 
# host_loc 존재?
df['host_location_boolean'] = df['host_location'].notnull().astype(int)
# host_loc in NY?
df['host_location_ny'] = df['host_location'].str.contains('New York', na=False).astype(int)



#rooms_info
# --- Personal preprocessing code ---
# Convert "beds" from float to int
# Replace missing or non-bed values with median (assumed 1)
df['beds'] = df['beds'].fillna(0).astype(int)
df['beds'] = df['beds'].replace(0, 1)

# Clean up "bathrooms", "bathrooms_text" column:
# - Replace invalid or missing values with median (assumed 1)
df['bathrooms'] = df['bathrooms'].fillna(0)

def parse_baths(text):
    if pd.isna(text):
        return np.nan
    s = str(text).lower()
    m = re.search(r'(\d+(\.\d+)?)', s)
    if m:
        return float(m.group(1))
    if 'half' in s:
        return 0.5
    return np.nan

df['bathrooms_parsed'] = df['bathrooms_text'].apply(parse_baths)
mask_mismatch = df['bathrooms_parsed'].notna() & (df['bathrooms'] != df['bathrooms_parsed'])
df.loc[mask_mismatch, 'bathrooms'] = df.loc[mask_mismatch, 'bathrooms_parsed']
df = df.drop(columns=['bathrooms_parsed'])

df['bathrooms_text'] = df['bathrooms_text'].fillna(0)

df['is_shared'] = df['bathrooms_text'] \
    .str.contains('shared', case=False, na=False)

df['is_private'] = ~df['is_shared']

w_private = 1.0   # 전용 욕실 가중치
w_shared  = 0.5   # 공용 욕실 가중치

df['bath_score_mul'] = (
    df['bathrooms'] * np.where(df['is_private'], w_private, w_shared)
)

df['bathrooms'] = df['bathrooms'].replace(0.00, 1)
df['bath_score_mul'] = df['bath_score_mul'].replace(0.00, 1)

# Clean up "room_type", "property_type" column:
#
def extract_structure(pt):
    pt_l = pt.strip().lower()
    if ' in ' in pt_l:
        return pt_l.split(' in ',1)[1].strip()
    if pt_l.startswith('entire '):
        return pt_l.replace('entire ','').strip()
    if pt_l.startswith('private room'):
        return pt_l.replace('private room','').strip()
    if pt_l.startswith('shared room'):
        return pt_l.replace('shared room','').strip()
    return pt_l

rt_cats = set(df['room_type'].str.strip().str.lower())
df['structure_type'] = df['property_type'].apply(lambda x: (
    x.strip().lower() if x.strip().lower() not in rt_cats
    else pd.NA
))

mask = df['structure_type'].notna()
df.loc[mask, 'structure_type'] = df.loc[mask, 'structure_type'].apply(extract_structure)

residential = {
    'rental unit','home','condo','townhouse','cottage',
    'bungalow','villa','vacation home','earthen home',
    'ranch','casa particular','tiny home','entire home/apt'
}
apartment_suite = {
    'guest suite','loft','serviced apartment','aparthotel',
    'private room'
}
hotel_lodging = {
    'hotel','boutique hotel','bed and breakfast',
    'resort','hostel','guesthouse','hotel room'
}

def map_category(row):
    pt = row['property_type'].strip().lower()
    rt = row['room_type'].strip().lower()
    st = row['structure_type']
    if rt in residential or pt in residential or (isinstance(st, str) and st in residential):
        return 'Residential'
    elif rt in apartment_suite or pt in apartment_suite or (isinstance(st, str) and st in apartment_suite):
        return 'Apartment_Suite'
    elif rt in hotel_lodging or pt in hotel_lodging or (isinstance(st, str) and st in hotel_lodging):
        return 'Hotel_Lodging'
    else:
        return 'Others'

df['structure_category'] = df.apply(map_category, axis=1)


for col in [
    'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
    'review_scores_checkin', 'review_scores_communication',
    'review_scores_location', 'review_scores_value'
]:
    df[col].fillna(df[col].mean(), inplace=True)
    df[col] = df[col].round(2)


availability_30/60/90/365 

예약가능여부가 예약으로 인한 불가인지 호스트가 의도적으로 막아둔 것으로 인한 불가인지 모호

네칼럼 비교했을때 30/60/90 데이터가 0인데 365만 값이 있는 경우가 많다
그래서 장기임대용 숙소여서 3개월이내로는 단기라고 보고 받지않나 했지만 maximum_nights/minimum_nights 확인했을때 아님
3개월이내 날짜는 모두 예약이 불가능하지만 장기적으로 봤을때 이후에는 예약이 가능하다?

In [ ]:
# 조건에 맞는 숙소 필터링
subset = df[
    (df['availability_30'] == 0) &
    (df['availability_60'] == 0) &
    (df['availability_90'] == 0) &
    (df['availability_365'] > 0)
].copy()

# 개수 확인
print(f"조건에 맞는 숙소 수: {len(subset)}")

# 주요 칼럼들 출력 (상위 10개)
display(subset[['availability_30', 'availability_60', 'availability_90', 'availability_365', 
              'minimum_nights', 'maximum_nights', 'last_review', 'host_since']].head(10))

In [ ]:
df['availability_365'].value_counts()
#365인 숙소: 2783 -> 예약없는 신규숙소이거나 수요가 매우 낮은 숙소일 가능성 
#0인 숙소 : 168 -> 지속적으로 예약이 되어있거나(단기숙박이 아닌 장기숙박일가능성높다),호스트가 의도적으로 막아둠 

시간경과에 따라서?
availability_365 에서 availability_30/60/90데이터를 뺐을때 단기적으로 얼마나 예약이 불가능해졌나-> 차이가 크면 예약이 가능한 날은 많았지만 30/60/90일 기준으로

335 (6057개 숙소): 표준적인 케이스/ 365일 열려있고 30일이내가능일도 30일인 경우
신규숙소이거나 낮은 수요/ 가격이 높거나 다른 이유로 경쟁력이 낮을 가능성

240 (1250개 숙소): 

In [ ]:
availability = df['availability_365'] - df['availability_30']
print(availability.value_counts().sort_values(ascending=False))
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
availability.plot(kind='hist', bins=40, edgecolor='black')
plt.title('최근 30일 가용일 수 분포')
plt.xlabel('최근 30일 중 예약 가능일 수')
plt.ylabel('숙소 수')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
availability_60 = df['availability_365'] - df['availability_60']
availability_90 = df['availability_365'] - df['availability_90']
print(availability_60.value_counts())
print(availability_90.value_counts())

availability_365 와 number_of_reviews_l365d
1년 중 오픈일수가 적지만 리뷰수가 많다면 인기있는 숙소(인기가 많아서 예약할수없는 숙소아닐까?)

In [ ]:
popular = df['number_of_reviews_ltm']/df['availability_365']
print(popular.value_counts().sort_values(ascending=False))

In [ ]:
# 리뷰가 0개이거나 365가 0일 가능성-> 리뷰는 0인데 365는 0이 아님 운영중이고 예약가능하지만 리뷰가 없음
zero_popular = df[popular == 0][['number_of_reviews_ltm', 'availability_365']]
print(zero_popular.head(10))
print(f"총 개수: {len(zero_popular)}")

# 0보다 크면 100일 오픈하면 1개 달릴까 말까? ->리뷰수가 availability 대비 매우 낮다
target_value = 0.01
tolerance = 0.001
near_target = df[(popular >= target_value - tolerance) & (popular <= target_value + tolerance)][['number_of_reviews_ltm', 'availability_365']]
print(near_target.head(10))
print(f"총 개수: {len(near_target)}")

# 1보다도 큰값은 availability가 적은데 리뷰가 많다.-> 8일 예약가능인데 리뷰가 17개/ 13일 예약가능 리뷰39개 (파티룸이나 단기로 빌리는 곳?)
high_popular = df[popular >= 1][['number_of_reviews_ltm', 'availability_365']]
print(high_popular.head(10))
print(f"총 개수: {len(high_popular)}")

In [ ]:
df.loc[popular.sort_values(ascending=False).index[:10], ['number_of_reviews_ltm', 'availability_365']]

availability_365
0일경우 예약가능한 날이 없다-> 이미 모든일정 예약 완료, 호스트의 차단(운영중단)
365일 경우 항상예약이 가능하다-> 신규숙소, 수요가 낮음, 장기임대로만 가능한 숙소 

availability_365-예약가능일수 estimated_occupancy_l365d-예약된일수 추정치?
availability_365 = 0, estimated_occupancy_l365d > 0 일경우 
예약가능한 날짜는 없는데 이전에 예약이 많았던 숙소 
- 예약이 다 차서 가능한 날짜가 없을 가능성
availability_365 > 0, estimated_occupancy_l365d = 0 일경우 
예약 가능한 날짜는 있으나, 지난 1년 동안 실제 예약은 하나도 안 됨
- 비인기 숙소이거나 신규호스트의 숙소 

In [ ]:
# 비교 대상
cond_1 = (df['availability_365'] == 0) & (df['estimated_occupancy_l365d'] > 0)
cond_2 = (df['availability_365'] > 0) & (df['estimated_occupancy_l365d'] == 0)

print("예약은 있었지만 현재 예약 가능일이 없는 숙소 (availability_365=0, occupancy>0):")
print(df[cond_1][['availability_365', 'estimated_occupancy_l365d']].head())
print(f"총 개수: {cond_1.sum()}개")

print("\n예약 가능하지만 지난 1년 예약이 없었던 숙소 (availability>0, occupancy=0):")
print(df[cond_2][['availability_365', 'estimated_occupancy_l365d']].head())
print(f"총 개수: {cond_2.sum()}개")

In [ ]:
from datetime import datetime

df['host_since'] = pd.to_datetime(df['host_since'], errors='coerce')
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

# 오늘 날짜 기준
today = pd.to_datetime("today")

# 3. 신규 호스트로 아직 예약 없음
new_no_booking = df[
    (df['host_since'] >= pd.to_datetime("2024-07-01")) &  # 최근 1년 내 가입
    (df['estimated_occupancy_l365d'] == 0)
]
print("최근 생성된 신규 숙소 중 예약 없는 경우:")
print(new_no_booking[['host_since', 'availability_365', 'estimated_occupancy_l365d']])